# Simple Diffusion

This Colab notebook provides a simple way for users to generate images using the [Stable Diffusion model](https://huggingface.co/CompVis/stable-diffusion), a text-to-image latent diffusion model created by the researchers and engineers from CompVis, Stability AI and LAION. This notebook is based on [HuggingFace's StableDiffusion Notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb), which you should use if you wish to have more fine-grained control of the model through code, rather than this notebook's text-box interface.

In [ ]:
#@title # Environment Setup

#@markdown ---
#@markdown ### HuggingFace Token:
#@markdown To download the model, you must accept it's licence on HuggingFace and enter an auth token for this notebook:
#@markdown 1. [Sign up for a Hugging Face account](https://huggingface.co/join).
#@markdown 2. Go to [the Stable Diffusion Model](https://huggingface.co/CompVis/stable-diffusion-v1-4) and accept the licence. Read through it first and make sure you agree to it!
#@markdown 3. Go to the [Access Tokens page](https://huggingface.co/settings/tokens) and create a token with "read" permissions.
#@markdown 4. Copy the token and paste it here. Make sure not to share this token with anyone.
token = "" #@param {type:"string"}

model = "CompVis/stable-diffusion-v1-4"

## Ensure a GPU is available
import os
if os.system("nvidia-smi") != 0:
  print("No GPU attached to this Colab instance. Please click Runtime>Change Runtime Type and change \"None\" to \"GPU\"")
  exit(1)

## Install necessary libraries
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

## Imports
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from huggingface_hub import notebook_login
from PIL import Image

## Download the Model
pipe = StableDiffusionPipeline.from_pretrained(model, revision="fp16", torch_dtype=torch.float16, use_auth_token=token)

## Move the model to the GPU
pipe = pipe.to("cuda")

In [ ]:
#@title # Image Generation 🎉🎉🎉

#@markdown ### Prompt:
#@markdown Set here what prompt you would like the images to be generated from
prompt = "" #@param {type:"string"}
#@markdown Image Width
width = 512 #@param {type:"integer"}
#@markdown Image Height
height = 512 #@param {type:"integer"}

#@markdown ---

#@markdown ### Advanced Model Parameters:
#@markdown Random seed passed to the generator. The same seed will always yield the same image output from the same prompt (assuming all other values are also the same)
random_seed = 0 #@param {type:"number"}
#@markdown Should the provided random seed (above) be used for the generator?
use_random_seed = False #@param {type:"boolean"}
#@markdown How many generation steps to run for each image. Fewer steps give less defined and realistic detail.
steps = 50 #@param {type:"integer"}
#@markdown Guidance scale
guidance_scale = 8 #@param {type:"number"}

# Random seeding
import torch
image_generator = torch.Generator("cuda")
if use_random_seed:
  image_generator.manual_seed(random_seed)
  print("Used provided random seed for the generator")
else:
  image_generator.seed()
  print("Used a new random seed for the generator")
print(f"Generator Seed: {image_generator.initial_seed()}")

# Guidance scale
print(f"Guidance Scale: {guidance_scale}")

# Generate Output Image
with autocast("cuda"):
  image = pipe(prompt, generator=image_generator, num_inference_steps=steps, guidance_scale=guidance_scale, height=height, width=width)["sample"][0]

image